<a href="https://colab.research.google.com/github/Guzhii/CAS_771_Project/blob/main/CIFAR-10-100/Framework_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import random
import numpy as np
from PIL import Image
import json
import os

# Global variables
device = "cuda"
GRAYSCALE = False
lr = 0.002
result_dir= 'results/'
noise_rate = 0
forget_rate = None
num_gradual = 40
exponent = 1
top_bn = False
dataset = 'cifar10'
NUM_CLASSES = 10
n_epoch=120
seed=1
print_freq=50
num_workers=4
num_iter_per_epoch=500
epoch_decay_start=80

# Seed
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

# Hyper Parameters
batch_size = 100
learning_rate = lr 

In [2]:
####################################################################
############### Noisy Loader  ######################################
####################################################################

def unpickle(file):
    import _pickle as cPickle
    with open(file, 'rb') as fo:
        dict = cPickle.load(fo, encoding='latin1')
    return dict


class cifar_dataset(Dataset):
    def __init__(self, dataset, root_dir, transform, mode, noise_file=''):

        self.transform = transform
        self.mode = mode
        self.transition = {0: 0, 2: 0, 4: 7, 7: 7, 1: 1, 9: 1, 3: 5, 5: 3, 6: 6,
                           8: 8}  # class transition for asymmetric noise for cifar10
        # generate asymmetric noise for cifar100
        self.transition_cifar100 = {}
        nb_superclasses = 20
        nb_subclasses = 5
        base = [1, 2, 3, 4, 0]
        for i in range(nb_superclasses * nb_subclasses):
            self.transition_cifar100[i] = int(base[i % 5] + 5 * int(i / 5))

        if self.mode == 'test':
            if dataset == 'cifar10':
                test_dic = unpickle('%s/test_batch' % root_dir)
                self.test_data = test_dic['data']
                self.test_data = self.test_data.reshape((10000, 3, 32, 32))
                self.test_data = self.test_data.transpose((0, 2, 3, 1))
                self.test_label = test_dic['labels']
            elif dataset == 'cifar100':
                test_dic = unpickle('%s/test' % root_dir)
                self.test_data = test_dic['data']
                self.test_data = self.test_data.reshape((10000, 3, 32, 32))
                self.test_data = self.test_data.transpose((0, 2, 3, 1))
                self.test_label = test_dic['fine_labels']
        else:
            train_data = []
            train_label = []
            if dataset == 'cifar10':
                for n in range(1, 6):
                    dpath = '%s/data_batch_%d' % (root_dir, n)
                    data_dic = unpickle(dpath)
                    train_data.append(data_dic['data'])
                    train_label = train_label + data_dic['labels']
                train_data = np.concatenate(train_data)
            elif dataset == 'cifar100':
                train_dic = unpickle('%s/train' % root_dir)
                train_data = train_dic['data']
                train_label = train_dic['fine_labels']
                # print(train_label)
                # print(len(train_label))
            train_data = train_data.reshape((50000, 3, 32, 32))
            train_data = train_data.transpose((0, 2, 3, 1))

            noise_label = json.load(open(noise_file, "r"))

            if self.mode == 'train':
                self.train_data = train_data
                self.noise_label = noise_label
                self.clean_label = train_label

    def __getitem__(self, index):
        if self.mode == 'train':
            img, target = self.train_data[index], self.noise_label[index]
            img = Image.fromarray(img)
            img = self.transform(img)
            return img, target, index
        elif self.mode == 'test':
            img, target = self.test_data[index], self.test_label[index]
            img = Image.fromarray(img)
            img = self.transform(img)
            return img, target

    def __len__(self):
        if self.mode != 'test':
            return len(self.train_data)
        else:
            return len(self.test_data)


class cifar_dataloader():
    def __init__(self, dataset, batch_size, num_workers, root_dir, noise_file=''):
        self.dataset = dataset
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.root_dir = root_dir
        self.noise_file = noise_file
        if self.dataset == 'cifar10':
            self.transform_train = transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
            self.transform_test = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
            ])
        elif self.dataset == 'cifar100':
            self.transform_train = transforms.Compose([
                transforms.RandomCrop(32, padding=4),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
            ])
            self.transform_test = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize((0.507, 0.487, 0.441), (0.267, 0.256, 0.276)),
            ])

    def run(self, mode):
        if mode == 'train':
            train_dataset = cifar_dataset(dataset=self.dataset,
                                          root_dir=self.root_dir, transform=self.transform_train, mode="train",
                                          noise_file=self.noise_file)
            trainloader = DataLoader(
                dataset=train_dataset,
                batch_size=self.batch_size,
                shuffle=True,
                num_workers=self.num_workers)
            return trainloader, np.asarray(train_dataset.noise_label), np.asarray(train_dataset.clean_label)

        elif mode == 'test':
            test_dataset = cifar_dataset(dataset=self.dataset,
                                         root_dir=self.root_dir, transform=self.transform_test, mode='test')
            test_loader = DataLoader(
                dataset=test_dataset,
                batch_size=self.batch_size,
                shuffle=False,
                num_workers=self.num_workers)
            return test_loader

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable


def conv3x3(in_planes, out_planes, stride=1):
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride, padding=1, bias=False)


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(in_planes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion * planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion * planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion * planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = conv3x3(3, 64) # number 1 indicate how many channels
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512 * block.expansion, num_classes)
        self.c_linear = nn.Linear(512 * block.expansion,1)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x, lin=0, lout=5):
        out = x
        if lin < 1 and lout > -1:
            out = self.conv1(out)
            out = self.bn1(out)
            out = F.relu(out)
        if lin < 2 and lout > 0:
            out = self.layer1(out)
        if lin < 3 and lout > 1:
            out = self.layer2(out)
        if lin < 4 and lout > 2:
            out = self.layer3(out)
        if lin < 5 and lout > 3:
            out = self.layer4(out)
        if lout > 4:
            out = F.avg_pool2d(out, 4)
            out = out.view(out.size(0), -1)
            feature = out
            out_c = self.c_linear(out)
            out = self.linear(out)
        return out, out_c

def resnet34():
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes=NUM_CLASSES)


# def test():
#     net = ResNet34()
#     y1, feature1,c1 = net(Variable(torch.randn(3, 3, 32, 32)))
#     y2, feature2,c2= net(Variable(torch.randn(3, 3, 32, 32)))
#     print(y1.size())
#     print(feature1.size())
#     print(c1.size())
#
# test()



model = resnet34().to(device)

In [10]:
# -*- coding:utf-8 -*-
import os
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms
import argparse, sys
import numpy as np
import datetime
import shutil


# load dataset
if dataset=='cifar10':
    input_channel=3
    num_classes=10
    top_bn = False
    epoch_decay_start = 80
    
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                            download=True, transform=transform)

    test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                            download=True, transform=transform)

    data_path = '/content/data/cifar-10-batches-py'  # path to the data file (don't forget to download the feature data and also put the noisy label file under this folder)

if dataset=='cifar100':
    input_channel=3
    num_classes=100
    top_bn = False
    epoch_decay_start = 100

    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    train_set = torchvision.datasets.CIFAR100(root='./data', train=True,
                                            download=True, transform=transform)

    test_set = torchvision.datasets.CIFAR100(root='./data', train=False,
                                            download=True, transform=transform)

    data_path = '/content/data/cifar-100-python'  # path to the data file (don't forget to download the feature data and also put the noisy label file under this folder)


# Data Loader (Input Pipeline)
print('loading dataset...')
loader = cifar_dataloader(dataset, batch_size=100,
                          num_workers=2,
                          root_dir=data_path,
                          noise_file='%s/cifar10_noisy_labels_task1.json' % (data_path))

train_loader, noisy_labels, clean_labels = loader.run('train')
test_loader = loader.run('test')

noise_or_not = np.transpose(noisy_labels)==np.transpose(clean_labels)
noise_rate = noise_or_not.tolist().count(False) / len(noise_or_not)



if forget_rate is None:
    forget_rate=noise_rate
else:
    forget_rate=forget_rate

# Adjust learning rate and betas for Adam Optimizer
mom1 = 0.9
mom2 = 0.1
alpha_plan = [learning_rate] * n_epoch
beta1_plan = [mom1] * n_epoch
for i in range(epoch_decay_start, n_epoch):
    alpha_plan[i] = float(n_epoch - i) / (n_epoch - epoch_decay_start) * learning_rate
    beta1_plan[i] = mom2

def adjust_learning_rate(optimizer, epoch):
    for param_group in optimizer.param_groups:
        param_group['lr']=alpha_plan[epoch]
        param_group['betas']=(beta1_plan[epoch], 0.999) # Only change beta1
        
# define drop rate schedule
rate_schedule = np.ones(n_epoch)*forget_rate
rate_schedule[:num_gradual] = np.linspace(0, forget_rate**exponent, num_gradual)

save_dir = result_dir +'/' +dataset+'/framework/'
if not os.path.exists(save_dir):
    os.system('mkdir -p %s' % save_dir)

model_str=dataset+'_framework_'+'_'+str(noise_rate)

txtfile=save_dir+"/"+model_str+str(num_gradual)+".txt"
nowTime=datetime.datetime.now().strftime('%Y-%m-%d-%H:%M:%S')
if os.path.exists(txtfile):
    os.system('mv %s %s' % (txtfile, txtfile+".bak-%s" % nowTime))


def accuracy(logit, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    output = F.softmax(logit, dim=1)
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0, keepdim=True)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res

Files already downloaded and verified
Files already downloaded and verified
loading dataset...


In [6]:
import torch
import torch.nn.functional as F

epi = 1e-12
ep_threshold = 60
momentum = 0.9
lossLamda = 0.5
regularization_strength = 0.1

if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True
    if torch.cuda.device_count() > 1:
        device = torch.device('cuda:0')
    else:
        device = torch.device('cuda')
else:
    device = torch.device('cpu')


class CAR(torch.nn.Module):
    def __init__(self, labels, num_classes):
        super(CAR, self).__init__()
        self.num_classes = NUM_CLASSES
        self.threshold_update = 0.0

        self.soft_labels = torch.zeros(labels.shape[0], num_classes, dtype=torch.float).cuda(non_blocking=True)
        self.soft_labels[torch.arange(labels.shape[0]), labels] = 1
        self.momentum = momentum
        self.beta = regularization_strength
        self.es = ep_threshold
        self.test = 0
        if torch.cuda.is_available():
          torch.backends.cudnn.benchmark = True
          if torch.cuda.device_count() > 1:
              self.device = torch.device('cuda:0')
          else:
              self.device = torch.device('cuda')
        else:
            device = torch.device('cpu')

    def forward(self, logits, confidence,  labels, index, lam, epoch):
        # sigmoid scale 0 to 1
        confidence = torch.sigmoid(confidence)

        output = F.softmax(logits, dim=1)
        eps = 1e-12
        output = torch.clamp(output, 0. + eps, 1. - eps)
        confidence = torch.clamp(confidence, 0. + eps, 1. - eps)
        one_hot = torch.zeros(len(labels), self.num_classes)
        one_hot[torch.arange(len(labels)), labels] = 1
        one_hot = one_hot.to(device)
        one_hot = torch.clamp(one_hot, min=1e-4, max=1.0)  # A=-4
        labels = labels.to(self.device)
        confidence = confidence.to(self.device)
        output = output.to(self.device)

        if epoch < ep_threshold:
            pred = confidence * output + (1 - confidence) * one_hot
            pred = torch.clamp(pred, min=1e-7, max=1.0)
            loss1 = -torch.mean(torch.sum(torch.log(pred) * one_hot, dim=1))
            rce = (-1 * torch.sum(pred * torch.log(one_hot), dim=1))
        else:
            if epoch % 10 == 0:
                temp_p = F.softmax(logits.detach(), dim=1)
                temp_p = temp_p.to(self.device)
                tp_f = confidence > self.threshold_update # only change the data has confidence >= threshold
                change_index = index[tp_f.view(tp_f.size()[0])]
                tp_f = tp_f.repeat(1, self.num_classes)
                self.soft_labels[change_index] = self.momentum * self.soft_labels[change_index] + (
                        1 - self.momentum) * temp_p[tp_f].view(-1, self.num_classes)
                self.soft_labels = torch.clamp(self.soft_labels, min=1e-4, max=1.0)
            pred = confidence * output + (1 - confidence) * self.soft_labels[index]
            pred = torch.clamp(pred, min=1e-7, max=1.0)
            loss1 = -torch.mean(torch.sum(torch.log(pred) * self.soft_labels[index], dim=1))
            rce = (-1 * torch.sum(pred * torch.log(self.soft_labels[index]), dim=1))

        loss2 = -torch.mean(torch.log(confidence))
        return loss1 + lam * loss2 + self.beta * rce.mean()
criterion = CAR(noisy_labels, NUM_CLASSES)

In [7]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np

def mixup_data(x, y, alpha=1.0, use_cuda=True):
    '''Returns mixed inputs, pairs of targets, and lambda'''
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    if use_cuda:
        index = torch.randperm(batch_size).cuda()
    else:
        index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam



def loss_coteaching(y_1, y_2, c_1, c_2, t_a, t_b, forget_rate, ind, noise_or_not, lam, ep_num):
    # Calculate and sort cross entropy losses for both models
    loss_1_a = F.cross_entropy(y_1, t_a, reduce = False)
    loss_1_b = F.cross_entropy(y_1, t_b, reduce = False)
    loss_1 = torch.add(torch.mul(loss_1_a, lam), torch.mul(loss_1_b, (1-lam)))
    ind_1_sorted = np.argsort(loss_1.cpu().data).cuda()
    loss_1_sorted = loss_1[ind_1_sorted]

    loss_2_a = F.cross_entropy(y_2, t_a, reduce = False)
    loss_2_b = F.cross_entropy(y_2, t_b, reduce = False)
    loss_2 = torch.add(torch.mul(loss_2_a, lam), torch.mul(loss_2_b, (1-lam)))
    ind_2_sorted = np.argsort(loss_2.cpu().data).cuda()
    loss_2_sorted = loss_2[ind_2_sorted]

    # Discard high loss samples based on forget rate
    remember_rate = 1 - forget_rate
    num_remember = int(remember_rate * len(loss_1_sorted))
    pure_ratio_1 = np.sum(noise_or_not[ind[ind_1_sorted.cpu()[:num_remember]]])/float(num_remember)
    pure_ratio_2 = np.sum(noise_or_not[ind[ind_2_sorted.cpu()[:num_remember]]])/float(num_remember)
    ind_1_update=ind_1_sorted[:num_remember]
    ind_2_update=ind_2_sorted[:num_remember]

    # exchange and update confidence adaptive losses of both models
    loss_1_a_update = criterion(y_1[ind_2_update], c_1[ind_2_update], t_a[ind_2_update], ind[ind_2_update], 50, ep_num)
    loss_1_b_update = criterion(y_1[ind_2_update], c_1[ind_2_update], t_b[ind_2_update], ind[ind_2_update], 50, ep_num)
    # loss_1_a_update = F.cross_entropy(y_1[ind_2_update], t_a[ind_2_update])
    # loss_1_b_update = F.cross_entropy(y_1[ind_2_update], t_b[ind_2_update])
    loss_1_update = torch.add(torch.mul(loss_1_a_update, lam), torch.mul(loss_1_b_update, (1-lam)))
    loss_2_a_update = criterion(y_2[ind_1_update], c_2[ind_1_update], t_a[ind_1_update], ind[ind_1_update], 50, ep_num)
    loss_2_b_update = criterion(y_2[ind_1_update], c_2[ind_1_update], t_b[ind_1_update], ind[ind_1_update], 50, ep_num)
    # loss_2_a_update = F.cross_entropy(y_2[ind_1_update], t_a[ind_1_update])
    # loss_2_b_update = F.cross_entropy(y_2[ind_1_update], t_b[ind_1_update])
    loss_2_update = torch.add(torch.mul(loss_2_a_update, lam), torch.mul(loss_2_b_update, (1-lam)))

    return loss_1_update, loss_2_update, pure_ratio_1, pure_ratio_2

In [11]:
## Train the Model
def train(train_loader,epoch, model1, optimizer1, model2, optimizer2):
    print('Training %s...' % model_str)
    pure_ratio_list=[]
    pure_ratio_1_list=[]
    pure_ratio_2_list=[]
    
    train_total=0
    train_correct=0 
    train_total2=0
    train_correct2=0 

    for i, (images, labels, indexes) in enumerate(train_loader):
        if i>num_iter_per_epoch:
            break
      
        images = Variable(images).cuda()
        labels = Variable(labels).cuda()

        #mixup images and labels
        images, labels_a, labels_b, lam = mixup_data(images, labels,
                                                       1, 1)
        images, labels_a, labels_b = map(Variable, (images,
                                                      labels_a, labels_b))
        
        # Get predictions and confidence value
        outputs1, confidence1 =model1(images)
        train_total+=outputs1.size(0)
        _, predicted = torch.max(outputs1.data, 1)
        train_correct+=(lam * predicted.eq(labels_a.data).cpu().sum().float()
                    + (1 - lam) * predicted.eq(labels_b.data).cpu().sum().float())

        outputs2, confidence2 = model2(images)
        train_total2+=outputs2.size(0)
        _, predicted = torch.max(outputs2.data, 1)
        train_correct2+=(lam * predicted.eq(labels_a.data).cpu().sum().float()
                    + (1 - lam) * predicted.eq(labels_b.data).cpu().sum().float())
        
        # Measure loss value 
        loss_1, loss_2, pure_ratio_1, pure_ratio_2 = loss_coteaching(outputs1.cpu(), outputs2.cpu(), confidence1.cpu(), confidence2.cpu(), labels_a.cpu(), labels_b.cpu(), rate_schedule[epoch], indexes, noise_or_not, lam, epoch)
        
        pure_ratio_1_list.append(100*pure_ratio_1)
        pure_ratio_2_list.append(100*pure_ratio_2)

        # Backward and optimizer
        optimizer1.zero_grad()
        loss_1.backward()
        optimizer1.step()
        optimizer2.zero_grad()
        loss_2.backward()
        optimizer2.step()
        if (i+1) % print_freq == 0:
            print ('Epoch [%d/%d], Iter [%d/%d] Training Accuracy1: %.4F, Training Accuracy2: %.4f, Loss1: %.4f, Loss2: %.4f, Pure Ratio1: %.4f, Pure Ratio2 %.4f' 
                  %(epoch+1, n_epoch, i+1, len(train_set)//batch_size, 100.*train_correct/train_total, 100.*train_correct2/train_total2, loss_1.data, loss_2.data, np.sum(pure_ratio_1_list)/len(pure_ratio_1_list), np.sum(pure_ratio_2_list)/len(pure_ratio_2_list)))

    train_acc1=float(train_correct)/float(train_total)
    train_acc2=float(train_correct2)/float(train_total2)
    return train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list

# Evaluate the Model
def evaluate(test_loader, model1, model2):
    print('Evaluating %s...' % model_str)
    model1.eval()    # Change model to 'eval' mode.
    correct1 = 0
    total1 = 0
    for images, labels in test_loader:
        images = Variable(images).cuda()
        logits1, _ = model1(images)
        outputs1 = F.softmax(logits1, dim=1)
        _, pred1 = torch.max(outputs1.data, 1)
        total1 += labels.size(0)
        correct1 += (pred1.cpu() == labels).sum()

    model2.eval()    # Change model to 'eval' mode 
    correct2 = 0
    total2 = 0
    for images, labels in test_loader:
        images = Variable(images).cuda()
        logits2, _ = model2(images)
        outputs2 = F.softmax(logits2, dim=1)
        _, pred2 = torch.max(outputs2.data, 1)
        total2 += labels.size(0)
        correct2 += (pred2.cpu() == labels).sum()
 
    acc1 = 100*float(correct1)/float(total1)
    acc2 = 100*float(correct2)/float(total2)
    return acc1, acc2


def main():
    print('building model...')
    cnn1 = resnet34().to(device)
    optimizer1 = torch.optim.Adam(cnn1.parameters(), lr=learning_rate)

    cnn2 = resnet34().to(device)
    optimizer2 = torch.optim.Adam(cnn2.parameters(), lr=learning_rate)

    mean_pure_ratio1=0
    mean_pure_ratio2=0

    with open(txtfile, "a") as myfile:
        myfile.write('epoch: train_acc1 train_acc2 test_acc1 test_acc2 pure_ratio1 pure_ratio2\n')


    epoch=0
    train_acc1=0
    train_acc2=0
    # evaluate models with random weights
    test_acc1, test_acc2=evaluate(test_loader, cnn1, cnn2)
    print('Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f %% Pure Ratio1 %.4f %% Pure Ratio2 %.4f %%' % (epoch+1, n_epoch, len(test_set), test_acc1, test_acc2, mean_pure_ratio1, mean_pure_ratio2))
    # save results
    with open(txtfile, "a") as myfile:
        myfile.write(str(int(epoch)) + ': '  + str(train_acc1) +' '  + str(train_acc2) +' '  + str(test_acc1) + " " + str(test_acc2) + ' '  + str(mean_pure_ratio1) + ' '  + str(mean_pure_ratio2) + "\n")

    # training
    for epoch in range(1, n_epoch):
        # train models
        cnn1.train()
        adjust_learning_rate(optimizer1, epoch)
        cnn2.train()
        adjust_learning_rate(optimizer2, epoch)
        train_acc1, train_acc2, pure_ratio_1_list, pure_ratio_2_list=train(train_loader, epoch, cnn1, optimizer1, cnn2, optimizer2)
        # evaluate models
        test_acc1, test_acc2=evaluate(test_loader, cnn1, cnn2)
        # save results
        mean_pure_ratio1 = sum(pure_ratio_1_list)/len(pure_ratio_1_list)
        mean_pure_ratio2 = sum(pure_ratio_2_list)/len(pure_ratio_2_list)
        print('Epoch [%d/%d] Test Accuracy on the %s test images: Model1 %.4f %% Model2 %.4f %%, Pure Ratio 1 %.4f %%, Pure Ratio 2 %.4f %%' % (epoch+1, n_epoch, len(test_set), test_acc1, test_acc2, mean_pure_ratio1, mean_pure_ratio2))
        with open(txtfile, "a") as myfile:
            myfile.write(str(int(epoch)) + ': '  + str(train_acc1) +' '  + str(train_acc2) +' '  + str(test_acc1) + " " + str(test_acc2) + ' ' + str(mean_pure_ratio1) + ' ' + str(mean_pure_ratio2) + "\n")

if __name__=='__main__':
    main()

building model...
Evaluating cifar10_framework__0.35962...
Epoch [1/120] Test Accuracy on the 10000 test images: Model1 9.8100 % Model2 10.0000 % Pure Ratio1 0.0000 % Pure Ratio2 0.0000 %
Training cifar10_framework__0.35962...


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


Epoch [2/120], Iter [50/500] Training Accuracy1: 10.5603, Training Accuracy2: 9.9766, Loss1: 3.3083, Loss2: 3.2801, Pure Ratio1: 64.4242, Pure Ratio2 64.3636
Epoch [2/120], Iter [100/500] Training Accuracy1: 11.3664, Training Accuracy2: 10.7590, Loss1: 3.0497, Loss2: 3.1227, Pure Ratio1: 64.3434, Pure Ratio2 64.1515
Epoch [2/120], Iter [150/500] Training Accuracy1: 12.2458, Training Accuracy2: 11.3676, Loss1: 3.0966, Loss2: 3.1100, Pure Ratio1: 64.2963, Pure Ratio2 64.1616


KeyboardInterrupt: ignored